In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tzfpy import get_tz
from pytz import timezone
import kds
import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import ydata_profiling as df_report

import seaborn as sns
import plotly.express as px
import matplotlib as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import sweetviz as sv
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h3
import keplergl

### Load data from private dataset and public dataset

In [ ]:
df_usage = pd.read_csv('data/usage_data_2023_05_01_to_15.csv')
display(df_usage.head(2))
df_airtraffic_trends = pd.read_csv('data/Airline_Transportation_Data.csv')
display(df_airtraffic_trends.head(2))

### Performing column re-names, EDA,Data Cleaning, Feature Engineering and Feature Selection 

In [ ]:

df_airtraffic_trends.rename(columns={'OBS_DATE': 'Month','ASM_US_D_I':'Aggr_seat_miles_domestic_and_international','LOAD_FACTOR_D_PCT_US':'Load_factor_domestic_pct_us'}, inplace=True)
df_airtraffic_trends.drop(['ID','ASM_D','ASM_I','LOAD_FACTOR_I_PCT_US'], axis=1, inplace=True)
df_airtraffic_trends['Month'] = pd.to_datetime(df_airtraffic_trends['Month'])
df_usage.columns = df_usage.columns.str.lower()
df_airtraffic_trends.columns = df_airtraffic_trends.columns.str.lower()
df_usage['usage_tstamp'] = pd.to_datetime(df_usage['usage_tstamp'])
df_usage['door_close_tstamp'] = pd.to_datetime(df_usage['door_close_tstamp'])
df_usage['door_open_plus15_tstamp'] = pd.to_datetime(df_usage['door_open_plus15_tstamp'])

# Based on several iterations of the EDA, it felt like I need to come up with a new feature which is percentage of flight completed 
# and need to find all time related variables in local time than in utc. Used lat,lon to convert time to local timezone.
# Dropped all rows with null vallues for lat,long

df_usage['pct_flight_completed'] = df_usage['time_into_flight']/((df_usage['door_open_plus15_tstamp'] - df_usage['door_close_tstamp']).dt.total_seconds() / 60)
df_usage.drop(['tail_id','door_close_tstamp','door_open_plus15_tstamp'], axis=1, inplace=True)
df_usage.dropna(subset=['latitude','longitude','seat_capacity','eco_seats','airline'],inplace=True)
df_usage['free_model'] = df_usage['airline'].apply(lambda x: 1 if x in ['DAL','JBU','JBU-R','QFA'] else 0)
# The initial data set had a lot of datapoints, so I sampled 1% of the data to do the EDA and build the model
df_usage_sampled = df_usage.sample(frac=0.5, random_state=42,replace=False)

display(df_airtraffic_trends.head(2))
display(df_usage_sampled.head(2))

In [ ]:
# Performing timezone trasnlations to get local time as well as hour of day and day of week

df_usage_sampled['usage_utc_hour'] = df_usage_sampled['usage_tstamp'].dt.hour
df_usage_sampled['usage_month']=df_usage_sampled['usage_tstamp'].dt.month_name()
df_usage_sampled['usage_utc_day']=df_usage_sampled['usage_tstamp'].dt.day_name()

def local_time_mapper(x):
    return get_tz(lat=x['latitude'], lng=x['longitude'])


df_usage_sampled['usage_tz'] = df_usage_sampled.apply(lambda x: local_time_mapper(x), axis=1)

# Throwing away few rows which had no timezone look up, very small numebr of rows
df_usage_sampled = df_usage_sampled[df_usage_sampled['usage_tz']!='']

# Converting utc time to local time based on the timezone and then getting the local hour of the day and day of the week

df_usage_sampled['usage_local_hour'] = df_usage_sampled.apply(lambda row: row['usage_tstamp'].tz_convert(timezone(row['usage_tz'])).hour, axis=1)
df_usage_sampled['usage_local_hour_shifted'] = df_usage_sampled['usage_local_hour'].apply(lambda x: (pd.Timedelta(hours=x) + pd.Timedelta(hours=-3)).components.hours)
df_usage_sampled['usage_local_day'] = df_usage_sampled.apply(lambda row: row['usage_tstamp'].tz_convert(timezone(row['usage_tz'])).strftime("%A"), axis=1)

display(df_usage_sampled.head(2))

In [ ]:
# Dropping un-wanted rows before doing one more final round of eda through sweetviz

df_usage_sampled.drop(['usage_tstamp','usage_tz'], axis=1, inplace=True)

df_usage_eda_report = sv.analyze(df_usage_sampled, target_feat='total_fl_mbps')
df_usage_eda_report.show_html('eda-viz/df_usage_per_tail_seatcapacity_eda_report.html')


In [ ]:
# Based on the output of the FInal run of eda, dropping unwanted columns as well as columns which are not available at the time of prediction ('total_rl_mbps')
df_usage_sampled.drop(columns=['usage_utc_day','usage_utc_hour','usage_local_hour','usage_month','total_rl_mbps'],inplace=True)
df_usage_sampled.info()

In [ ]:
# Converting local_hour and pct flight completed to categorical variables based on the output of final eda run
df_usage_sampled['usage_local_hour_shifted'] = df_usage_sampled['usage_local_hour_shifted'].astype('str')
df_usage_sampled['pct_flight_completed'] = round((100*df_usage_sampled['pct_flight_completed']/10),0).astype('str')

In [ ]:
#throwing outliers in total_fl_mbps which are between 0.1 and  99.5th percentile
df_usage_sampled = df_usage_sampled[ ( df_usage_sampled['total_fl_mbps']<df_usage_sampled['total_fl_mbps'].quantile(0.995)) & (df_usage_sampled['total_fl_mbps']>df_usage_sampled['total_fl_mbps'].quantile(0.001)) ]

In [ ]:
# Applying column transformer to encode categorical variables and scale numeric variables

# Get columns of integer or float type
numeric_cols = df_usage_sampled.select_dtypes(include=['int64', 'float64']).columns
print(numeric_cols)
# Get columns of object type
object_cols = df_usage_sampled.select_dtypes(include=['object']).columns
print(object_cols)

# Categorical feature columns to one-hot encode
categorical_cols = ['airline','flight_phase','usage_local_day','pct_flight_completed','usage_local_hour_shifted','pct_flight_completed']
# Numerical feature columns to standard scale
numerical_cols = ['altitude','seat_capacity','eco_seats']

# Create transformers for one-hot encoding and standard scaling
column_trans = ColumnTransformer(
    transformers=[
        ('ohe',OneHotEncoder(sparse=False), categorical_cols),
        ('std_scaled',StandardScaler(), numerical_cols),
    ],
    remainder='passthrough',
    )
column_trans.set_output(transform='pandas')
transformed_data = column_trans.fit_transform(df_usage_sampled)
df_usage_column_transformed = pd.DataFrame(transformed_data)

df_usage_column_transformed.rename(columns={'remainder__total_fl_mbps':'total_fl_mbps'}, inplace=True)
display(df_usage_column_transformed.head(2))

In [ ]:
# X_before_poly = df_usage_column_transformed.drop('total_fl_mbps', axis=1) 
# display(X_before_poly.shape)
# display(X_before_poly.head())
# poly_features = PolynomialFeatures(degree=2)
# X_poly = poly_features.fit_transform(X_before_poly)
# X = pd.DataFrame(X_poly, columns=poly_features.get_feature_names_out(X_before_poly.columns))

# display(X.shape)
# display(X.head())
display(df_usage_column_transformed.info())


In [ ]:
# Split the df into train test split with 80:20 ratio 

X = df_usage_column_transformed.loc[:, df_usage_column_transformed.columns != 'total_fl_mbps']
y = df_usage_column_transformed['total_fl_mbps']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defining a set of regression models to be used for hyper parameter tuning

regressors = {
    'Linear Regression': (LinearRegression(), {}),
    'Ridge Regression': (Ridge(fit_intercept=False), {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}),
    'Lasso Regression': (Lasso(), {'alpha': [0.00001,0.0001,0.001,0.01, 0.1, 1.0, 10.0, 100.0]}),
    'Random Forest Regressor': (RandomForestRegressor(n_jobs=-1), {'n_estimators': [ 50, 100, 200], 'max_depth': [5, 7, 10, 15, 20, 40]}),
    #'Support Vector Regressor': (SVR(), {'C': [1, 10], 'kernel': ['linear', 'rbf']})
}

best_models = []
# Perform grid search for each regressor
for regressor_name, (regressor, param_grid) in regressors.items():
    grid_search = GridSearchCV(regressor, param_grid, scoring='r2', cv=5, verbose=2)
    grid_search.fit(X_train, y_train)

    # Get the best estimator and make predictions
    best_regressor = grid_search.best_estimator_
    y_pred = best_regressor.predict(X_test)

    # Calculate and print the R2 score

    r2 = r2_score(y_test, y_pred)
    print(f"{regressor_name}: Best Parameters: {grid_search.best_params_}, R2 Score: {r2}")
    best_models.append({'regressor':regressor_name,'model':best_regressor})


In [ ]:
display(best_models[3]['regressor'])
display(best_models[3]['model'].feature_importances_)
importances = best_models[3]['model'].feature_importances_

In [ ]:
# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]
sorted_importances = importances[indices]
sorted_features = X_test.columns[indices]

# Plot the feature importances
plt.figure(figsize=(8, 4))
plt.title("Feature Importance")
plt.bar(range(X_test.shape[1]), sorted_importances, align="center")
plt.xticks(range(X_test.shape[1]), sorted_features, rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error, mean_absolute_error
# Print mean absolute percentage error (MAPE), R2 score, and mean absolute error (MAE)


best_regressor = best_models[3]['model']
y_pred = best_regressor.predict(X_test)

# Calculate and print the MAPE score
mape = round(mean_absolute_percentage_error(y_test, y_pred),2)
mae = round(mean_absolute_error(y_test, y_pred),2)
mse = round(mean_squared_error(y_test, y_pred),2)
r2 = r2_score(y_test, y_pred)
print("Best Regressor's Scores: MAPE : {mape}, MAE : {mae}, MSE : {mse}, R2 : {r2}".format(mape=mape,mse=mse,mae=mae,r2=r2))

In [ ]:
# Plot the predicted vs actual values
plt.figure(figsize=(10, 6))
plt.title("Predicted vs Actual")
plt.scatter(y_test, y_pred)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()
